In [1]:
import torch
import torch.nn.functional as F
import networkx as nx
import scipy as sp
import scipy.io
import numpy as np

In [2]:
from torch_geometric.nn import GCNConv
from torch_geometric.utils.convert import from_networkx
from torch_geometric.transforms import RandomNodeSplit
from torch_geometric.data import Data

### Transform to generate train, val, test data

In [3]:
transform = RandomNodeSplit('train_rest', key='y')

### Matrix Market Graph (bcspwr07)

In [5]:
bcspwr07 = sp.io.mmread("./matrix_market_graphs/bcspwr07.mtx")
mmg2 = nx.from_scipy_sparse_matrix(bcspwr07)

/tmp/ipykernel_182028/1057088530.py:2: DeprecationWarning: 

The scipy.sparse array containers will be used instead of matrices
in Networkx 3.0. Use `from_scipy_sparse_array` instead.
  mmg2 = nx.from_scipy_sparse_matrix(bcspwr07)


#### convert scipy coo_matrix format to Torch.tensor

In [6]:
values = bcspwr07.data
indices = np.vstack((bcspwr07.row, bcspwr07.col))
i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = bcspwr07.shape

In [7]:
sparse_tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape))
print(sparse_tensor.to_sparse_csr())

tensor(crow_indices=tensor([   0,    3,    5,  ..., 5817, 5821, 5824]),
       col_indices=tensor([   0,   32,  143,  ..., 1595, 1610, 1611]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]), size=(1612, 1612),
       nnz=5824, layout=torch.sparse_csr)


#### create data object from Networkx

In [8]:
data = from_networkx(mmg2)
data.x = sparse_tensor.to_sparse_csr()
data.y = torch.randint(0,10, [data.num_nodes,])  # random y values since this dataset is not curated
data = transform(data)

In [9]:
data.num_features = 1612
data.num_classes = 10

In [10]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(data.num_features, 16)
        self.conv2 = GCNConv(16, data.num_classes)
        
    def forward(self, data):
         x, edge_index = data.x, data.edge_index
         x = self.conv1(x, edge_index)
         x = F.relu(x)
         x = F.dropout(x, training=self.training)
         x = self.conv2(x, edge_index)
         return F.log_softmax(x, dim=1)

### CUDA errors for sparse_csr data

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model = Net().to(device)
data = data.to(device)

NotImplementedError: Could not run 'aten::empty_strided' with arguments from the 'SparseCsrCUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::empty_strided' is only available for these backends: [CPU, CUDA, Meta, BackendSelect, Python, Named, Conjugate, Negative, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradLazy, AutogradXPU, AutogradMLC, AutogradHPU, AutogradNestedTensor, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, Tracer, UNKNOWN_TENSOR_TYPE_ID, Autocast, Batched, VmapMode].

CPU: registered at /opt/conda/conda-bld/pytorch_1640811803361/work/build/aten/src/ATen/RegisterCPU.cpp:18433 [kernel]
CUDA: registered at /opt/conda/conda-bld/pytorch_1640811803361/work/build/aten/src/ATen/RegisterCUDA.cp

In [14]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = Net().to(device)
data = data.to(device)

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()
for epoch in range(200):
     optimizer.zero_grad()
     out = model(data)
     loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
     #loss.backward()
     optimizer.step()

### Backprop causes an error

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()
for epoch in range(200):
     optimizer.zero_grad()
     out = model(data)
     loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
     loss.backward()
     optimizer.step()

NotImplementedError: Could not run 'aten::empty.memory_format' with arguments from the 'SparseCsrCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::empty.memory_format' is only available for these backends: [CPU, CUDA, Meta, MkldnnCPU, SparseCPU, SparseCUDA, BackendSelect, Python, Named, Conjugate, Negative, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradLazy, AutogradXPU, AutogradMLC, AutogradHPU, AutogradNestedTensor, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, Tracer, UNKNOWN_TENSOR_TYPE_ID, Autocast, Batched, VmapMode].

CPU: registered at /opt/conda/conda-bld/pytorch_1640811803361/work/build/aten/src/ATen/RegisterCPU.cpp:18433 [kernel]
CUDA: registered at /opt/conda/conda-bld/pytorch_164081180

### Low accuracy due to randomly generated y values

In [16]:
model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.1020
